In [1]:
import re
import collections
import emoji
import pandas as pd
import numpy as np

# processing of text
import spacy
from autocorrect import Speller
from stopwords import stopwords

# models
import torch
# from lazypredict.Supervised import LazyClassifier
import xgboost
from model_functions import train_validation_test_split
from model_functions import split_train_val_test
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
# load and download
spell = Speller('pl')
lemma_spacy = spacy.load('pl_spacy_model')

In [3]:
class Preprocessing:
    def __init__(self):
        self.x_raw = None
        self.y = None
        self.x_train = None
        self.x_val = None
        self.x_test = None
        self.y_train = None
        self.y_val = None
        self.y_test = None

    def load_data(self):
        data_poleval_raw = pd.read_csv(
            'Labeled_data/converted_label_poleval.txt',
            error_bad_lines=False, sep=",")
        data_poleval_test_raw = pd.read_csv(
            'Labeled_data/converted_label_poleval_test.txt',
            error_bad_lines=False, sep=",")
        data_github_raw = pd.read_csv('Labeled_data/labeled_dataset.txt',
                                      error_bad_lines=False, sep="|")

        data_poleval = data_poleval_raw.copy(deep=True)
        data_poleval_test = data_poleval_test_raw.copy(deep=True)
        data_github = data_github_raw.copy(deep=True)

        conected_data = pd.concat([data_poleval,
                                   data_poleval_test,
                                   data_github.drop(['No', 'Yes'],
                                                    axis=1)],
                                  axis=0,
                                  ignore_index=True)
        self.x_raw = conected_data['Comment']
        self.y = conected_data['Kind of offensive language']

    def remove_quoting_comments(self):
        pattern = r'^RT.*'
        remove = self.x_raw.str.contains(pattern)
        self.x_raw = self.x_raw[~remove].reset_index(drop=True)
    
    def demojize(self):
        self.x_raw = self.x_raw.apply(lambda x: emoji.demojize(x,  delimiters=("~~", "~~")))
    
    def clean_text(self, stopwords_remove = False):
        # remove of @name
        pattern = re.compile(r'@\w+[\s]*')
        self.x_raw = self.x_raw.str.replace(pattern, '')

        # split emoji
        pattern = re.compile(r"~{2}")
        self.x_raw = self.x_raw.str.replace(pattern, ' ')

        # remove of links https
        pattern = re.compile(r"https?[:\/\/]+[a-zA-Z0-9.\-\/?=_~:#%]+")
        self.x_raw = self.x_raw.str.replace(pattern, '')

        # removal of punctuations and numbers
        pattern = re.compile(r'[^_ąćęłńóśźżĄĆĘŁŃÓŚŹŻa-zA-Z\s]')
        self.x_raw = self.x_raw.str.replace(pattern, '')

        # remove more than one space
        pattern = re.compile(r'\s+')
        self.x_raw = self.x_raw.str.replace(pattern, ' ')

        # remove beginning and ending task space
        pattern = re.compile(r'^\s+|\s+?$')
        self.x_raw = self.x_raw.str.replace(pattern, '')

        # remove of capitalization
        self.x_raw = self.x_raw.str.lower()
        
        # remove stopwords
    def stopwords_remove(self):
        self.x_raw = self.x_raw.apply(lambda x: x.split())
        self.x_raw = self.x_raw.apply(
            lambda x: [item for item in x if
                       item not in stopwords and len(item) >= 3])
        for i in range(len(self.x_raw)):
            self.x_raw[i] = ' '.join(self.x_raw[i])
                
    def lemmatize_text(self):
        self.x_raw = self.x_raw.apply(lambda x: [token.lemma_ for token in lemma_spacy(x)])
        
    def correct_typo_words(self):
        self.dict_words_place = collections.defaultdict(list)
        for _, row_list in self.x_raw.items():
            for place_in_row in range(len(row_list)):
                self.dict_words_place[row_list[place_in_row]].append((_, place_in_row))
    
        with open('slowa.txt', encoding='utf-8') as file:
            contents = file.read()
            for word, place_word in self.dict_words_place.items():
                if word in contents:
                    continue
                else:
                    for token in lemma_spacy(spell(word)):
                        correct = token.lemma_
                    for number in place_word:
                        self.x_raw[number[0]][number[1]] = correct
    
    def token_join(self):
            self.x_raw = self.x_raw.apply(lambda x: ' '.join(x))
    
    def write_to_file(self):
        self.x_raw.to_csv('clean_x_raw.csv', index=False)
        self.y.to_csv('clean_y.csv', index=False)

In [4]:
data = Preprocessing()

In [5]:
data.load_data()
data.remove_quoting_comments()
data.demojize()
data.clean_text()
data.stopwords_remove()
data.lemmatize_text()
data.correct_typo_words()
data.token_join()
data.write_to_file()

# Model

In [ ]:
data = train_validation_test_split(conected_data)

In [ ]:
data

In [ ]:
test = data[data['split'] =='train'].iloc[:10][['Comment', 'Kind of offensive language']]
test['Comment'] = preprocess_text(test['Comment'], stopwords_remove=True)
# test['Comment'] = test['Comment'].apply(lambda x: x.split())
test

In [ ]:
test['Comment'] = lemmatize_text(test['Comment'])

In [25]:
test

NameError: name 'test' is not defined

In [ ]:
# test = data[data['split'] =='train'][['Comment', 'Kind of offensive language']]
# test['Comment'] = test['Comment'].apply(lambda x: x.split())
# test

In [ ]:
# test['Comment'] = test['Comment'].apply(lambda x: [token.lemma_ for token in lemma_spacy(x)])

In [19]:
conected_data = split_train_val_test()

In [24]:
conected_data['Comment'] = conected_data['Comment'].apply(lambda x: ' '.join(x))

KeyError: 'Comment'

In [ ]:
data[data['split'] =='val'].iloc[:10]['Kind of offensive language']

In [ ]:
X_train = data[data['split'] =='train'].iloc[:10][['Comment']]
y_train =  data[data['split'] =='train'].iloc[:10]['Kind of offensive language']
X_test = data[data['split'] =='val'].iloc[:10][['Comment']]
y_test =  data[data['split'] =='val'].iloc[:10]['Kind of offensive language']
tfidf_vectorizer = TfidfVectorizer()

# TF-IDF feature matrix
X_train = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
X_train = data[data['split'] =='train'].iloc[:10][['Comment']]
y_train =  data[data['split'] =='train'].iloc[:10]['Kind of offensive language']
X_test = data[data['split'] =='val'].iloc[:10][['Comment']]
y_test =  data[data['split'] =='val'].iloc[:10]['Kind of offensive language']


tfidf_vectorizer = TfidfVectorizer()

# TF-IDF feature matrix
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)


# print(tfidf_vectorizer.get_feature_names())

In [ ]:
X_train = data[data['split'] =='train']['Comment']
y_train =  data[data['split'] =='train']['Kind of offensive language']
X_test = data[data['split'] =='val']['Comment']
y_test =  data[data['split'] =='val']['Kind of offensive language']

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# print(vectorizer.get_feature_names())

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
result = ''
the_best_result = ''
class Modeling:
    """Modeling and presentation of results"""

    def __init__(self, model, X_train, X_test, title):
        """Inicjalization"""

        self.model = model
        self.title = title
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.result = ''

    def fit_predict(self):
        """Function to train, predict our model and create roc curve"""
        self.classifier = self.model
        self.classifier.fit(self.X_train, self.y_train)
        self.y_pred = self.classifier.predict(self.X_test)
        self.y_pred_proba = self.classifier.predict_proba(self.X_test)[:, 1]

        self.y_train_proba = self.classifier.predict_proba(self.X_train)[:, 1]

    def print_results(self):
        """Function to print our result"""
        self.accuracy = round(accuracy_score(y_test, self.y_pred), 4)
        self.f1 = round(f1_score(y_test, self.y_pred, average='weighted'), 4)
        self.recall = round(recall_score(y_test, self.y_pred, average='weighted'), 4)
#         self.log_loss = round(log_loss(y_test, self.y_pred_proba), 4)

        print(f'Results for {self.title}:')
        print(f'{self.title} accuracy: {self.accuracy}')
        print(f'{self.title} f-score: {self.f1}')
        print(f'{self.title} recall: {self.recall}')
#         print(f'{self.title} log_loss: {self.log_loss}')

    def add_to_table(self):
        """Function to add our result to dataframe to compare all"""
        global result
        if len(result) == 0:
            result = {self.title: [self.accuracy, self.f1, self.recall]}
            result = pd.DataFrame(result, index=['Accuracy', 'F-score',
                                                 'Recall'])
        else:
            conact = {self.title: [self.accuracy, self.f1, self.recall,
                                   self.log_loss]}
            conact = pd.DataFrame(conact, index=['Accuracy', 'F-score',
                                                 'Recall'])
            result = pd.concat([result, conact], axis=1)

    def plot_confusion_matrix(self):
        """plot confusion matrix"""
        plt.figure(figsize=(10, 10), facecolor='w')
        sns.heatmap(confusion_matrix(y_test, self.y_pred), annot=True,
                    fmt='.0f',
                    cbar=False,
                    vmax=confusion_matrix(y_test, self.y_pred).max(),
                    vmin=0, cmap='Blues')
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(f'Confusion matrix for {self.title}')

    def plot_confusion_matrix_percent(self):
        """Plot confusion matrix with part of 1 value"""
        plt.figure(figsize=(10, 10), facecolor='w')
        cm = confusion_matrix(y_test, self.y_pred)
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        df_cm = pd.DataFrame(cm_norm)
        sns.heatmap(df_cm, annot=True, cmap="Blues", cbar=False)
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(f'Confusion matrix for {self.title}')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc, log_loss

In [ ]:
log_reg = Modeling(RandomForestClassifier(), 
                   X_train, X_test, 
                   'Logistic Regression')
log_reg.fit_predict()
log_reg.print_results()
log_reg.add_to_table()

In [ ]:
log_reg.plot_confusion_matrix_percent()

In [ ]:
result

In [ ]:
result = ''
the_best_result = ''
class Modeling:
    """Modeling and presentation of results"""
    
    def __init__(self, model, X_train, X_test, title, pca=False):
        """Inicjalization"""
        
        self.model = model
        self.title = title
        self.X_sample = X_train
        self.y_sample = y_train
        self.X_test = X_test
        self.pca = pca
        self.result = ''
        
    def sample(self, sampling):
        self.sample = sampling
        self.X_sample, self.y_sample = self.sample.fit_resample(
            self.X_sample, self.y_sample)

    def fit_predict(self):
        """Function to train, predict our model and create roc curve"""
        self.classifier = self.model
        self.classifier.fit(self.X_sample, self.y_sample)
        self.y_pred = self.classifier.predict(self.X_test)
        self.y_pred_proba = self.classifier.predict_proba(self.X_test)[:, 1]
        if self.pca:
            self.y_train_proba = self.classifier.predict_proba(X_train_pca)[:, 1]
        else:
            self.y_train_proba = self.classifier.predict_proba(X_train)[:, 1]
        

    def print_results(self):
        """Function to print our result"""
        self.accuracy = round(accuracy_score(y_test, self.y_pred, 'weighted'), 4)
        self.f1 = round(f1_score(y_test, self.y_pred, average='weighted'), 4)
        self.recall = round(recall_score(y_test, self.y_pred, average='weighted'), 4)
#         self.log_loss = round(log_loss(y_test, self.y_pred_proba), 4)
        
        print(f'Results for {self.title}:')
        print(f'{self.title} accuracy: {self.accuracy}')
        print(f'{self.title} f-score: {self.f1}')
        print(f'{self.title} recall: {self.recall}')
#         print(f'{self.title} log_loss: {self.log_loss}')

    def add_to_table(self):
        """Function to add our result to dataframe to compare all"""
        global result
        if len(result) == 0:
            result  = {self.title: [self.accuracy, self.f1, self.recall
                                    ]}
            result = pd.DataFrame(result, index=['Accuracy', 'F-score', 
                                                 'Recall'])
        else:
            conact = {self.title: [self.accuracy, self.f1, self.recall, 
                                   ]}
            conact = pd.DataFrame(conact, index=['Accuracy', 'F-score', 
                                                 'Recall'])
            result = pd.concat([result, conact], axis=1) 

    def plot_confusion_matrix(self):
        """plot confusion matrix"""
        plt.figure(figsize=(10,10), facecolor='w')
        sns.heatmap(confusion_matrix(y_test, self.y_pred), annot=True, fmt='.0f', 
                    cbar=False, vmax=confusion_matrix(y_test, self.y_pred).max(), 
                    vmin=0, cmap='Blues')
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(f'Confusion matrix for {self.title}')
    
    def plot_confusion_matrix_percent(self):
        """Plot confusion matrix with part of 1 value"""
        plt.figure(figsize=(10,10), facecolor='w')
        cm = confusion_matrix(y_test, self.y_pred)
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        df_cm = pd.DataFrame(cm_norm)
        sns.heatmap(df_cm, annot=True, cmap="Blues", cbar=False)
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(f'Confusion matrix for {self.title}')

In [ ]:
log_reg = Modeling(RandomForestClassifier(), 
                   X_train, X_test, 
                   'Logistic Regression')
log_reg.fit_predict()
log_reg.print_results()
log_reg.add_to_table()

In [ ]:
log

In [ ]:
# SMOTE for Logistic Regression

log_reg_smote = Modeling(LogisticRegression(), 
                         X_train, X_test,
                         'Logistic Regression SMOTE')
log_reg_smote.sample(SMOTE(sampling_strategy='minority'))
log_reg_smote.fit_predict()
log_reg_smote.print_results()
log_reg_smote.plot_confusion_matrix()
log_reg_smote.add_to_table()

In [ ]:
! pip install imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
# SMOTE for Logistic Regression

log_reg_smote = Modeling(SVC(probability=True), 
                         X_train, X_test,
                         'Logistic Regression SMOTE')
log_reg_smote.sample(SMOTE(sampling_strategy='minority'))
log_reg_smote.fit_predict()
log_reg_smote.print_results()
log_reg_smote.plot_confusion_matrix()
log_reg_smote.add_to_table()

In [ ]:
print(classification_report(y_test,log_reg.y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
bayes = Modeling(MultinomialNB(), 
                         X_train, X_test,
                         'Bayes')
bayes.sample(SMOTE(sampling_strategy='minority'))
bayes.fit_predict()
bayes.print_results()
bayes.plot_confusion_matrix()
bayes.add_to_table()

In [ ]:
from xgboost import XGBClassifier

In [ ]:
oversample = SMOTE()
X_train_s, y_train_s = oversample.fit_resample(X_train, y_train)
train = xgboost.DMatrix(X_train_s, label=y_train_s)
test = xgboost.DMatrix(X_test, label=y_test)

In [ ]:
param = {'max_depth':50, 'eta':0.5, 'objective': 'multi:softmax', 'num_class': 3}
epochs=100
bst = xgboost.train(param, train, epochs)
# make prediction
preds = bst.predict(test)
print(classification_report(y_test,preds))

In [ ]:
from gensim.models import Word2Vec